# 2.2.0 Running the CoxLab PredNet

The original PredNet wasn't straight forward to  get working. See [note-1.0](https://github.com/apra93/predictive-event-segmentation/blob/master/docs/source/notes/note-1.0-Running-the-CoxLab-PredNet.md) for details on getting their scripts running. This WB is for meesing with those scripts' innards.

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and other non-code related info
%watermark -n -m -g -b -t -h

Thu Aug 27 2020 15:12:55 

compiler   : GCC 7.5.0
system     : Linux
release    : 5.4.0-42-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
host name  : apra-x3
Git hash   : 4a276e8b21f39b812271fa7087ff1bd6d130e11d
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode
%matplotlib inline

## Conda Env


In [4]:
!conda list

# packages in environment at /home/apra/miniconda3/envs/prednet:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
_tflow_select             2.1.0                       gpu    anaconda
absl-py                   0.9.0            py37hc8dfbb8_1    conda-forge
argon2-cffi               20.1.0                   pypi_0    pypi
astor                     0.8.1              pyh9f0ad1d_0    conda-forge
async-generator           1.10                     pypi_0    pypi
attrs                     20.1.0             pyh9f0ad1d_0    conda-forge
babel                     2.8.0                    pypi_0    pypi
backcall                  0.2.0                    pypi_0    pypi
beautifulsoup4            4.9.1                      py_1    conda-forge
bleach                    3.1.5                    pypi_0    pypi
brotlipy                  0.

## Imports

In [29]:
from pprint import pprint
import os
import numpy as np
from six.moves import cPickle
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from keras import backend as K
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Flatten
from PIL import Image, ImageOps

%aimport prevseg.index
import prevseg.index as index

%aimport prednet.kitti_settings
import prednet.kitti_settings as ks
%aimport prednet.prednet_base
import prednet.prednet_base as pn
%aimport prednet.data_utils
import prednet.data_utils as utils

# Keep track of versions of everything
%watermark -v -iv

prevseg    0+untagged.52.g4a276e8.dirty
numpy      1.19.1
prednet    0+untagged.70.gc5532a0.dirty
matplotlib 3.3.1
PIL.Image  7.2.0
CPython 3.7.8
IPython 7.17.0


## Running the Evaluate Kitti Code

See `prednet.kitti_evaluate.py` for reference.

In [6]:
n_plot = 40
batch_size = 10
nt = 10
weights_file = str(ks.WEIGHTS_DIR / 'tensorflow_weights/prednet_kitti_weights.hdf5')
json_file = str(ks.WEIGHTS_DIR / 'prednet_kitti_model.json')
test_file = str(ks.DATA_DIR / 'X_test.hkl')
test_sources = str(ks.DATA_DIR / 'sources_test.hkl')

In [7]:
# Load trained model
f = open(json_file, 'r')
json_string = f.read()
f.close()
pprint(json_string)

('{"class_name": "Model", "keras_version": "2.0.6", "config": {"layers": '
 '[{"class_name": "InputLayer", "config": {"dtype": "float32", '
 '"batch_input_shape": [null, 10, 3, 128, 160], "name": "input_1", "sparse": '
 'false}, "inbound_nodes": [], "name": "input_1"}, {"class_name": "PredNet", '
 '"config": {"dtype": "float32", "trainable": true, "pixel_max": 1.0, '
 '"Ahat_filt_sizes": [3, 3, 3, 3], "return_state": false, "R_filt_sizes": [3, '
 '3, 3, 3], "unroll": false, "batch_input_shape": [null, null, null], '
 '"LSTM_inner_activation": "hard_sigmoid", "output_mode": "error", "stateful": '
 'false, "error_activation": "relu", "A_activation": "relu", "A_filt_sizes": '
 '[3, 3, 3], "stack_sizes": [3, 48, 96, 192], "name": "prednet_1", '
 '"R_stack_sizes": [3, 48, 96, 192], "data_format": "channels_first", '
 '"implementation": 0, "extrap_start_time": null, "go_backwards": false, '
 '"return_sequences": true, "LSTM_activation": "tanh"}, "inbound_nodes": '
 '[[["input_1", 0, 0, {}]]]

In [8]:
train_model = model_from_json(json_string, custom_objects = {'PredNet': pn.PredNet})
train_model.load_weights(weights_file)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [9]:
# Create testing model (to output predictions)
layer_config = train_model.layers[1].get_config()
layer_config['output_mode'] = 'prediction'
data_format = layer_config['data_format'] if 'data_format' in layer_config \
    else layer_config['dim_ordering']
test_prednet = pn.PredNet(weights=train_model.layers[1].get_weights(), 
                          **layer_config)

In [10]:
input_shape = list(train_model.layers[0].batch_input_shape[1:])
input_shape[0] = nt
inputs = Input(shape=tuple(input_shape))
predictions = test_prednet(inputs)
test_model = Model(inputs=inputs, outputs=predictions)

In [11]:
print(test_file, test_sources, nt, data_format, sep='\n')

/home/apra/work/prednet/prednet/kitti_data/X_test.hkl
/home/apra/work/prednet/prednet/kitti_data/sources_test.hkl
10
channels_first


In [12]:
test_generator = utils.SequenceGenerator(test_file, 
                                         test_sources, 
                                         nt, 
                                         sequence_start_mode='unique', 
                                         data_format=data_format)
X_test = test_generator.create_all()

In [13]:
X_test.shape

(83, 10, 3, 128, 160)

In [14]:
X_hat = test_model.predict(X_test, batch_size)

In [15]:
# Presumably for plotting?
if data_format == 'channels_first':
    X_test = np.transpose(X_test, (0, 1, 3, 4, 2))
    X_hat = np.transpose(X_hat, (0, 1, 3, 4, 2))

In [16]:
print(X_test.shape, X_hat.shape, sep='\n')

(83, 10, 128, 160, 3)
(83, 10, 128, 160, 3)


In [17]:
mse_model = np.mean((X_test[:, 1:] - X_hat[:, 1:])**2)  # look at all timesteps except the first
mse_prev = np.mean((X_test[:, :-1] - X_test[:, 1:])**2)

In [18]:
print("Model MSE: %f" % mse_model)
print("Previous Frame MSE: %f" % mse_prev)

Model MSE: 0.006995
Previous Frame MSE: 0.021246


## Getting Model Representations

In [19]:
layer_config

{'name': 'prednet_1',
 'trainable': True,
 'batch_input_shape': (None, None, None),
 'dtype': 'float32',
 'return_sequences': True,
 'return_state': False,
 'go_backwards': False,
 'stateful': False,
 'unroll': False,
 'implementation': 0,
 'stack_sizes': [3, 48, 96, 192],
 'R_stack_sizes': [3, 48, 96, 192],
 'A_filt_sizes': [3, 3, 3],
 'Ahat_filt_sizes': [3, 3, 3, 3],
 'R_filt_sizes': [3, 3, 3, 3],
 'pixel_max': 1.0,
 'error_activation': 'relu',
 'A_activation': 'relu',
 'LSTM_activation': 'tanh',
 'LSTM_inner_activation': 'hard_sigmoid',
 'data_format': 'channels_first',
 'extrap_start_time': None,
 'output_mode': 'prediction'}

In [24]:
# Create testing model (to output predictions)
layer_config = train_model.layers[1].get_config()
X_Rs = []

test_generator = utils.SequenceGenerator(test_file, 
                                         test_sources, 
                                         nt, 
                                         sequence_start_mode='unique', 
                                         data_format=data_format)
X_test = test_generator.create_all()

for i in range(len(layer_config['R_stack_sizes'])):

    layer_config['output_mode'] = 'R' + str(i)
    data_format = layer_config['data_format'] if 'data_format' in layer_config \
        else layer_config['dim_ordering']
    test_prednet = pn.PredNet(weights=train_model.layers[1].get_weights(), 
                              **layer_config)

    input_shape = list(train_model.layers[0].batch_input_shape[1:])
    input_shape[0] = nt
    inputs = Input(shape=tuple(input_shape))
    R_outs = test_prednet(inputs)
    test_model = Model(inputs=inputs, outputs=R_outs)

    X_Rs.append(test_model.predict(X_test, batch_size))

In [25]:
X_Rs[-1].shape

(83, 10, 192, 16, 20)

## Matching the Schapiro Images to CoxLabs PredNet Input

In [30]:
X_test.shape

(83, 10, 3, 128, 160)

Each image needs to be of shape (3, 128, 160).

In [76]:
# Borrowing some code from ``prevseg/schapiro/resnet_embedding.py``

sch_shape = (128, 128)
pn_shape = (128, 160)

# Load and Resize
paths_fractals = list(index.DIR_SCH_FRACTALS.iterdir())
list_fractals = [Image.open(str(path)) for path in paths_fractals]
_ = [img.load() for img in list_fractals]

# Remove the alpha channel
list_fractals_no_alpha = [Image.new("RGB", pn_shape, (0,0,0))
                          for _ in range(len(list_fractals))]
_ = [bk.paste(img, (0, int((pn_shape[1]-sch_shape[1])/2 - 1)), mask=img.split()[3])
     for bk, img in zip(list_fractals_no_alpha, list_fractals)]

list_arrays = [np.moveaxis(np.array(img), (0,1,2), (2,1,0))
               for img in list_fractals_no_alpha]

In [80]:
# Save the embeddings
save_path = index.DIR_SCH / 'abstract_discs_resized_128_160/'
if not save_path.exists():
    save_path.mkdir()

for arr, path in zip(list_arrays, paths_fractals):
    # Turn to an array and reorder the dims
    np.save(str(save_path / path.stem), arr)
    arr2 = np.load(str(save_path / (path.stem + '.npy')))

    assert arr.shape == arr2.shape
    assert arr.mean() == arr2.mean()

This was made into the script `prevseg/schapiro/coxlab_resize.py`.